In [11]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
 
from torchvision import models

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [21]:
learning_rate = 0.001
training_epochs = 50
batch_size = 200

In [14]:
mnist_train = dsets.FashionMNIST(root='MNIST_data/',
                          transform=transforms.ToTensor(),
                          train=True,
                          download=True)

mnist_test = dsets.FashionMNIST(root='MNIST_data/',
                         transform=transforms.ToTensor(),
                         train=False,
                         download=True)

In [22]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)


In [23]:
class FashionCNN(torch.nn.Module):
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2)
        )
        
        self.fc1 = torch.nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = torch.nn.Dropout2d(0.25)
        self.fc2 = torch.nn.Linear(in_features=600, out_features=120,bias=True)
        self.fc3 = torch.nn.Linear(in_features=120, out_features=10,bias=True)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

In [24]:
model=FashionCNN().to(device)

In [25]:
loss = torch.nn.CrossEntropyLoss().to(device)   
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [26]:
#train my model
total_batch = len(train_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.464546204
[Epoch:    2] cost = 0.298165083
[Epoch:    3] cost = 0.265751749
[Epoch:    4] cost = 0.238167778
[Epoch:    5] cost = 0.22546865
[Epoch:    6] cost = 0.204383492
[Epoch:    7] cost = 0.194329113
[Epoch:    8] cost = 0.186424658
[Epoch:    9] cost = 0.172681496
[Epoch:   10] cost = 0.161944211
[Epoch:   11] cost = 0.155397281
[Epoch:   12] cost = 0.149740443
[Epoch:   13] cost = 0.135792598
[Epoch:   14] cost = 0.127824202
[Epoch:   15] cost = 0.124285571
[Epoch:   16] cost = 0.121711224
[Epoch:   17] cost = 0.112372473
[Epoch:   18] cost = 0.105257578
[Epoch:   19] cost = 0.0988533199
[Epoch:   20] cost = 0.0969537273
[Epoch:   21] cost = 0.0910127386
[Epoch:   22] cost = 0.0829189792
[Epoch:   23] cost = 0.0833749995
[Epoch:   24] cost = 0.0764121562
[Epoch:   25] cost = 0.075586535
[Epoch:   26] cost = 0.0785605013
[Epoch:   27] cost = 0.0680619329
[Epoch:   28] cost = 0.065795891
[Epoch:   29] cost = 0.060555182

In [28]:
accuracy = 0
total_batch = len(test_loader)

with torch.no_grad():
  model.eval()

  for X, Y in test_loader:
    Y = Y.to(device)
    X = X.to(device)


    prediction = model(X)
    correct_prediction = torch.argmax(prediction, 1) == Y
    accuracy += correct_prediction.float().mean()

print('Accuracy:', accuracy.item()/total_batch)

Accuracy: 0.909599838256836
